In [1]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from war_functions.pecota_tables import *
from cluster_luck_functions.cluster_luck_hitting import *
from cluster_luck_functions.cluster_luck_pitching import *
from cluster_luck_functions.cluster_luck_combined import *
from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *
from daily_adjustments.todays_game_info import *
from daily_adjustments.starting_rotations_WAR import *
from daily_adjustments.adjusted_war_today import *
from odds_and_other_projections import *

In [2]:
bets = pd.read_csv('past_bets/base/bets_20220918.csv', index_col = 0)

In [3]:
bets = bets[bets.Home_Team != 'Los Angeles Angels']
bets.reset_index(drop = True, inplace = True)
bets

,Home_Team,Away_Team,Home_Prob,Away_Prob,Home_ML,Away_ML,Home_ML_Prob,Away_ML_Prob,Home_Diff,Away_Diff,Home_KC,Away_KC,Home_Bet,Away_Bet
0,Tampa Bay Rays,Texas Rangers,0.642458,0.357542,-200.0,170.0,0.666667,0.370370,-0.024208,-0.012829,0.000000,0.000000,0.000000,0.000000
1,Boston Red Sox,Kansas City Royals,0.780069,0.219931,-195.0,165.0,0.661017,0.377358,0.119052,-0.157428,0.035120,0.000000,4188.063012,0.000000
2,Atlanta Braves,Philadelphia Phillies,0.861259,0.138741,-255.0,215.0,0.718310,0.317460,0.142949,-0.178719,0.042289,0.000000,5042.915929,0.000000
3,Washington Nationals,Miami Marlins,0.454153,0.545847,155.0,-180.0,0.392157,0.642857,0.061996,-0.097010,0.010199,0.000000,1216.252477,0.000000
4,Toronto Blue Jays,Baltimore Orioles,0.661628,0.338372,-200.0,170.0,0.666667,0.370370,-0.005038,-0.031999,0.000000,0.000000,0.000000,0.000000
5,New York Mets,Pittsburgh Pirates,0.765487,0.234513,-450.0,360.0,0.818182,0.217391,-0.052695,0.017122,0.000000,0.002188,0.000000,260.889607
6,Cleveland Guardians,Minnesota Twins,0.645910,0.354090,-105.0,-115.0,0.512195,0.534884,0.133715,-0.180794,0.027412,0.000000,3268.785917,0.000000
7,Houston Astros,Oakland Athletics,0.936344,0.063656,-330.0,275.0,0.767442,0.266667,0.168902,-0.203010,0.042722,0.000000,5094.565803,0.000000
8,Milwaukee Brewers,New York Yankees,0.273962,0.726038,155.0,-180.0,0.392157,0.642857,-0.118195,0.083181,0.000000,0.023291,0.000000,2777.373064
9,St. Louis Cardinals,Cincinnati Reds,0.813474,0.186526,-275.0,230.0,0.733333,0.303030,0.080141,-0.116505,0.030053,0.000000,3583.767328,0.000000


In [4]:
bets.to_csv('past_bets/base/bets_20220918.csv')

In [17]:
# Fixing error

# Necessary parameters
current_year = dt.date.today().year
today = str(dt.date.today()).replace('-', '')
today_date = dt.date.today()
yesterday = today_date - dt.timedelta(days=1)
yesterday_string = str(yesterday)
yesterday_string = yesterday_string.replace('-', '')
team_map = {
    'Giants' : 'San Francisco Giants',
    'Astros' : 'Houston Astros',
    'Brewers': 'Milwaukee Brewers',
    'Dodgers': 'Los Angeles Dodgers',
    'Rays' : 'Tampa Bay Rays',
    'Red Sox': 'Boston Red Sox',
    'White Sox': 'Chicago White Sox',
    'Padres': 'San Diego Padres',
    'Athletics': 'Oakland Athletics',
    'Yankees' : 'New York Yankees',
    'Mariners': 'Seattle Mariners',
    'Reds': 'Cincinnati Reds',
    'Blue Jays' : 'Toronto Blue Jays',
    'Mets' : 'New York Mets',
    'Phillies': 'Philadelphia Phillies',
    'Angels': 'Los Angeles Angels',
    'Braves': 'Atlanta Braves',
    'Cardinals': 'St. Louis Cardinals',
    'Tigers': 'Detroit Tigers',
    'Cubs': 'Chicago Cubs',
    'Indians': 'Cleveland Indians',
    'Guardians' : 'Cleveland Guardians',
    'Nationals': 'Washington Nationals',
    'Rockies': 'Colorado Rockies',
    'Marlins': 'Miami Marlins',
    'Royals': 'Kansas City Royals',
    'Twins': 'Minnesota Twins',
    'Pirates': 'Pittsburgh Pirates',
    'Rangers': 'Texas Rangers',
    'Orioles': 'Baltimore Orioles',
    'Diamondbacks': 'Arizona Diamondbacks',
    'D-backs' : 'Arizona Diamondbacks'
}

# Getting yesterdays results from CBS
link = 'https://www.cbssports.com/mlb/scoreboard/' + yesterday_string + '/'
tables = pd.read_html(link)
results_table = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Winner'])
team_regex = r'\D*'
for table in tables:
    if list(table.columns) == ['Unnamed: 0', 'R', 'H', 'E']:

        # Getting team names
        team_away_string = table.iloc[0,0]
        team_away_list = re.findall(team_regex, team_away_string)
        team_away = team_away_list[0]
        
        team_home_string = table.iloc[1,0]
        team_home_list = re.findall(team_regex, team_home_string)
        team_home = team_home_list[0]
        
        # Getting score and determining winner
        runs_away = table.iloc[0,1]
        runs_home = table.iloc[1,1]
        if runs_away>runs_home:
            winner = team_away
        else:
            winner = team_home
        
        # Appending to results table
        series = pd.Series([team_home, team_away, winner], index = results_table.columns)
        results_table = results_table.append(series, ignore_index = True)        
    else:
        continue
for column in list(results_table.columns):
    results_table[column] = results_table[column].apply(lambda x: team_map[x])

results_table

,Home_Team,Away_Team,Winner
0,New York Yankees,Tampa Bay Rays,New York Yankees
1,Chicago Cubs,San Francisco Giants,San Francisco Giants
2,Oakland Athletics,Chicago White Sox,Chicago White Sox
3,Kansas City Royals,Detroit Tigers,Detroit Tigers
4,Baltimore Orioles,Boston Red Sox,Boston Red Sox
5,Philadelphia Phillies,Washington Nationals,Philadelphia Phillies
6,Miami Marlins,New York Mets,New York Mets
7,Texas Rangers,Toronto Blue Jays,Toronto Blue Jays
8,Milwaukee Brewers,Cincinnati Reds,Milwaukee Brewers
9,Minnesota Twins,Cleveland Guardians,Cleveland Guardians


In [6]:
tables

[     Unnamed: 0   R   H  E
 0     Rays78-59   3   7  0
 1  Yankees84-56  10  15  1,
                                          0
 0  W: J. Taillon (13-4)L: C. Kluber (10-8),
            0      1
 0  Box Score  Recap,
     Unnamed: 0  R   H  E
 0  Giants66-73  5  11  1
 1    Cubs58-81  2   7  0,
                                                    0
 0  W: L. Webb (13-8)L: M. Stroman (3-7)S: C. Dova...,
            0      1
 0  Box Score  Recap,
        Unnamed: 0   R   H  E
 0  White Sox72-68  10  20  2
 1  Athletics50-90   2   3  0,
                                       0
 0  W: L. Lynn (6-5)L: A. Martinez (4-4),
            0      1
 0  Box Score  Recap,
     Unnamed: 0  R   H  E
 0  Tigers54-85  8  11  1
 1  Royals56-84  4   4  1,
                                         0
 0  W: M. Manning (2-2)L: J. Heasley (3-8),
            0      1
 0  Box Score  Recap,
      Unnamed: 0   R   H  E
 0  Red Sox68-72  17  21  1
 1  Orioles73-66   4   8  0,
                                        0